In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from src.Utils import normalize_data, unnormalize_data, plot_image, plot_value_array

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return dic

def load_data(path=None):
    if path is None:
        cifar = keras.datasets.cifar10
        return cifar.load_data()
    else:
        d=[]
        l=[]
        for i in range(5):
            batch = unpickle(path + "/data_batch_" + str(i+1))
            d.append(batch[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1))
            l.append(np.asarray(batch[b'labels']).reshape((10000,1)))
        train_images = np.concatenate( [ d[i] for i in range(5) ])
        train_labels = np.concatenate( [ l[i] for i in range(5) ])
        batch = unpickle(path + "/test_batch")
        test_images = batch[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
        test_labels = np.asarray(batch[b'labels']).reshape((10000,1))
        return (train_images, train_labels), (test_images, test_labels)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()
#(train_images, train_labels), (test_images, test_labels) = load_data("data/cifar-10-batches-py")

In [ ]:
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [ ]:
train_images, mu_X, sigma_X = normalize_data(train_images)
test_images, mu_X, sigma_X = normalize_data(test_images, mu_X, sigma_X)

In [ ]:
optimizer_names = ['SGD', 'Adagrad', 'RMSprop', 'Adam']
optimizer_list = [ 
    optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001), 
    optimizers.Adagrad(lr=0.01),
    optimizers.RMSprop(),
    optimizers.Adam()
    ]
model = []

for i, name in enumerate(optimizer_names):
    model.append(keras.Sequential([
        keras.layers.Conv2D(16, kernel_size=3, padding='same'),
        keras.layers.LeakyReLU(alpha=0.01),
        keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid', data_format=None),
        keras.layers.Flatten(),
        keras.layers.Dense(1024),
        keras.layers.LeakyReLU(alpha=0.01),
        keras.layers.Dense(64),
        keras.layers.LeakyReLU(alpha=0.01),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ]))
    model[i].compile(optimizer=optimizer_list[i], 
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

In [ ]:
for i in range(len(optimizer_names)):
    model[i].fit(train_images, train_labels, batch_size=1024, epochs=5)

In [ ]:
for i in range(len(optimizer_names)):
    test_loss, test_acc = model[i].evaluate(test_images, test_labels)
    print('Test accuracy:', test_acc)

In [ ]:
for i in range(len(optimizer_names)):
    plt.plot(range(0,5), model[i].history.history['loss'])
    plt.xlim(0,5)
    plt.ylim(0,2)
    plt.legend(['SGD', 'Adagrad', 'RMSprop', 'Adam'], loc='upper right')